In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

c:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


cuda:0


In [2]:
# !pip install accelerate==0.19.0
# !pip install bitsandbytes
# !pip install peft

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
np.bool = bool
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from Datasetloader import Datasetloader 
import evaluate
rouge = evaluate.load('rouge')

# import pandas as pd
# import regex as re
# import os
# from datasets import load_dataset, Dataset, DatasetDict
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import Compose, Resize, ToTensor
# from PIL import Image
# import wandb


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

def get_trainable_params(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

max_lecture_len = 450

cuda:0


# dataset loading

In [3]:
model_name="bigscience/bloom-560m"
# model_name="openlm-research/open_llama_3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def generate_prompt(lecture, question_group, max_lecture_len, max_question_group_len):
    prompt = "Here is a lecture. Write a few exercises for self study. \n\n###Lecture:\n\n" + lecture[:max_lecture_len] + "\n\n###Questions:\n\n" + question_group[:max_question_group_len]
    return prompt

In [4]:
loader = Datasetloader()
train_lectures, val_lectures, train_dataset, val_dataset = loader.load_all_data_for_hf_GPT(
    tokenizer, 
    generate_prompt,
    model_name=model_name, 
    max_lecture_len=max_lecture_len, 
    max_question_group_len=500, 
    tokenizer_max_len=256, 
    cache=False, 
    # question_type="is_exam", 
    # max_questions_returned=400
    )

Index(['Book', 'Chapter', 'section_title', 'is_question', 'text', 'summary',
       'question_answers', 'truefalse', 'fillblank', 'mc', 'concept',
       'open_ended', 'chapter', 'key_terms', 'index', 'references',
       'num_sentences', 'num_words', 'length'],
      dtype='object')
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for question: 03_Principles of Management
Missing lecture for

Map (num_proc=4):   0%|          | 0/3480 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3480 [00:00<?, ? examples/s]

In [5]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3480
})

In [6]:
loader.get_info(train_lectures)

2784
lecture: Stages of Moral Development How do ethics evolve over time? Psychologist Lawrence Kohlberg suggests that there are six distinct stages of moral development and that some individuals move further along
questions: 
Know the three levels and six stages of moral development suggested by Kohlberg.
Describe famous corporate scandals.
Understand how Bill 198 of 2002 provides a check on corporate ethical behaviour in Canada.
Know the dimensions of corporate social performance tracked by KLD.

lecture: Giant mammals lived during the ice ages because conditions were favorable. Earth's climate has been warmer and colder in Earth history, but mostly it's been warmer. Climate Change in Earth History Cli
questions: 
Explain how Earth's climate has changed in the past
How important is climate in the history of life
Dinosaurs lived a long time, geologically speaking, in part because the weather was favorable to them

lecture: Annuities A common financial goal is to be comfortable in reti

In [ ]:
print(str(train_dataset[1]["input_ids"]))
filler_tokens = []
for thing in train_dataset:
    filler_tokens.append(str(thing["input_ids"]).count("250680"))

print(np.array(filler_tokens).mean())
print(np.array(filler_tokens).min())
print(np.array(filler_tokens).max())


# model training

In [ ]:
from peft import LoraConfig, get_peft_model,PeftModel, PeftConfig

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)


def prepare_model(model):
  for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
      # cast the small parameters (e.g. layernorm) to fp32 for stability
      # param.data = param.data.to(torch.float32)
      param.data = param.data.to(torch.float16)
    else:
      # cast the large parameters to fp16 for memory savings
      param.data = param.data.to(torch.float16)

  model.gradient_checkpointing_enable()  # reduce number of stored activations
  model.enable_input_require_grads()
  model.lm_head = CastOutputToFloat(model.lm_head)

  return model


def prepare_model2(model):
  config = LoraConfig( r=4, #attention heads
      lora_alpha=32, #alpha scaling
      lora_dropout=0.1, #dropouts
      bias="none",
      task_type="CAUSAL_LM" # set this for CAUSAL LANGUAGE MODELS (like Bloom, LLaMA) or SEQ TO SEQ (like FLAN, T5)
  )

  model = get_peft_model(model, config)

  return model



In [ ]:
# print(type(model))
# print(dir(model))
# print(model.config)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    #load_in_8bit=True, 
    #device_map='auto',
)

load_saved_model = False
# load_saved_model = True

if load_saved_model:
    
    model = model.to(device)
    model = prepare_model(model)
    model = prepare_model2(model)
    model = PeftModel.from_pretrained(model, './trained_models/')
    model.eval()
    get_trainable_params(model)

else:

    model = prepare_model(model)
    model = prepare_model2(model)
    model.train()
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer:transformers.Trainer = transformers.Trainer(
        model=model, 
        train_dataset=train_dataset,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=6, 
            gradient_accumulation_steps=6,
            warmup_steps=1, 
            max_steps=0, 
            num_train_epochs=1,
            learning_rate=2e-4, 
            fp16=True,
            logging_steps=1, 
            output_dir='outputs',
            report_to="none",
            lr_scheduler_type="constant",
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    get_trainable_params(model)
    trainer.train()
    model.save_pretrained('./trained_models/')

model = model.to(device)
model:transformers.PreTrainedModel


# model testing

In [ ]:
with torch.no_grad():
  # check if the model works
  for id in range(15,25):

    example_batch = tokenizer(generate_prompt(val_lectures[id][0], "", max_lecture_len, 9999), return_tensors='pt')
    example_batch = {k: v.to(device) for k, v in example_batch.items()}

    with torch.cuda.amp.autocast():
      output_tokens = model.generate(**example_batch, max_new_tokens=30, temperature=0.1, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1)

    print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True), '\n\n')
    for question in val_lectures[id][1]:
      print('original:', question)

    print("\n\n\n\n\n")

In [ ]:
# To evaluate the model, this function computes the perplexity of the model when it generates the question.
def check_perplexity(model, tokenizer):
    with torch.no_grad():
        perplexity_total = 0
        perplexity_values = []
        for id,(lecture,questions) in enumerate(val_lectures):
            question_group = "\n".join(questions)
            prompt = generate_prompt(lecture, question_group, max_lecture_len, 400)
            # print(prompt);break
            prompt_tokenized = tokenizer.encode(prompt)

            input = torch.tensor(prompt_tokenized).unsqueeze(0).to(device)
            labels = torch.tensor(prompt_tokenized).unsqueeze(0).to(device)
            #Only the question matters
            labels[:,:-len(tokenizer.encode(question))] = -100

            # print("input: " + str(input))
            # print("labels: " + str(labels))
            
            outputs = model(input_ids=input, labels=labels)
            loss = outputs.loss
            
            perplexity = torch.exp(loss).item()
            perplexity_total += perplexity
            perplexity_values.append(perplexity)
            print("question " + str(id+1) + " of " + str(len(val_lectures)) + " done. perplexity: " + str(perplexity)[:6] + "\n"
                   + " average perplexity: " + str(perplexity_total / (id+1))[:6] + "\n"
                   + " median perplexity: " + str(np.median(perplexity_values))[:6] + "\n\n"
                   )
            
        return perplexity_total / len(val_lectures)


perplexity = check_perplexity(model, tokenizer)
print(perplexity)

In [ ]:
# from evalpackage.qrelscore import QRelScore

# # instantiate the scorer class by loading required pre-trained language models
# scorer = QRelScore()

# context_str = 'The fight scene finale between Sharon and the character played by Ali Larter, from the movie Obsessed, won the 2010 MTV Movie Award for Best Fight.'
# candidate_str = 'Which award did the fight scene between Sharon and the role of Ali Larter win?'

# # a list of input passages (contexts)
# gts = [ context_str ]
# # a list of generated questions (candidates)
# res = [ candidate_str ]

# # a list of relevance scores
# scores = scorer.compute_score_flatten(gts, res)
# print(scores)

In [ ]:
# To evaluate the model, this function computes the rouge score of the model's generated question and the original questions
def check_rouge(model, tokenizer):
    with torch.no_grad():
        rouge1_total = 0
        rouge2_total = 0
        rougeL_total = 0
        rougeLsum_total = 0
        qrelscore_total = 0
        for id,(lecture,questions) in enumerate(val_lectures):
            
            question_group = "\n".join(questions)
            prompt = generate_prompt(lecture,"",max_lecture_len,400)
            prompt_tokenized = tokenizer(prompt, return_tensors='pt')
            # print(prompt_tokenized)
            input = {k: v.to(device) for k, v in prompt_tokenized.items()}

            with torch.cuda.amp.autocast():
                output_tokens = model.generate(**input, max_new_tokens=30, temperature=0.1, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1)

            output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
            output = output.replace(prompt, "")

            rouge_current = rouge.compute(predictions=[output], references=[question_group], use_aggregator=True)
            qrelscore_current = scorer.compute_score_flatten([question_group[:400]], [output[:400]])

            rouge1_total += rouge_current['rouge1']
            rouge2_total += rouge_current['rouge2']
            rougeL_total += rouge_current['rougeL']
            rougeLsum_total += rouge_current['rougeLsum']
            qrelscore_total += qrelscore_current[0]

            print("processing lecture " + str(id+1) + " of " + str(len(val_lectures))  + "\n"
                    + " rouge1 average score: " + str(rouge1_total / (id+1))[:6]  + "\n"
                    + " rouge2 average score: " + str(rouge2_total / (id+1))[:6]  + "\n"
                    + " rougeL average score: " + str(rougeL_total / (id+1))[:6]  + "\n"
                    + " rougeLsum average score: " + str(rougeLsum_total / (id+1))[:6]  + "\n"
                    + " qrelscore average score: " + str(qrelscore_total / (id+1))[:6]  + "\n\n"
                  )
            # print(output)
            # print(question_group)

            # break




check_rouge(model, tokenizer)


In [ ]:
# lecture = "13.3   Political Behavior in OrganizationsHow do managers cope effectively with organizational politics?Closely related to the concept of power is the equally important topic of politics. In any discussion of the exercise of power—particularly in intergroup situations—a knowledge of basic political processes is essential. We will begin our discussion with this in mind. Next, on the basis of this analysis, we will consider political strategies for acquiring, maintaining, and using power in intergroup relations. Finally, we look at ways to limit the impact of political behavior in organizations.What Is Politics?Perhaps the earliest definition of politics was offered by Lasswell, who described it as who gets what, when, and how.15 Even from this simple definition, one can see that politics involves the resolution of differing preferences in conflicts over the allocation of scarce and valued resources. Politics represents one mechanism to solve allocation problems when other mechanisms, s"

# def check_perplexity_once(question):
#     prompt = generate_prompt(lecture,question,max_lecture_len=600)
#     prompt_tokenized = tokenizer.encode(prompt)

#     input = torch.tensor(prompt_tokenized).unsqueeze(0).to(device)
#     labels = torch.tensor(prompt_tokenized).unsqueeze(0).to(device)
#     labels[:,:-len(tokenizer.encode(question))] = -100
#     outputs = model(input_ids=input, labels=labels)
#     loss = outputs.loss
#     print(torch.exp(loss).item())


# check_perplexity_once("What are the components that constitute power in organizations?")
# check_perplexity_once("diuoamhdvk sadad84271942179423790470sdsaghj")
# check_perplexity_once("deez nuts")
# check_perplexity_once("How do managers cope effectively with organizational politics?")
# check_perplexity_once("What is politics?")
# check_perplexity_once("What are the political strategies for acquiring, maintaining, and using power in intergroup relations")
# check_perplexity_once("What are the political strategies for acquiring, maintaining, and using power in intergroup relations?")
# check_perplexity_once("What do politics represent?")
# check_perplexity_once("I refuse to do this task")
# print(val_questions[65][0])
# check_perplexity_once(val_questions[65][0])



In [ ]:
# # evaluate the generated questions with another model

# evaluator = pipeline('text-generation', model='linkanjarad/Bloom-Alpaca-560m', device=device, tokenizer=tokenizer, framework="pt", use_fast=True)
# # convert evaluator to fp16 for faster inference
# evaluator.model = evaluator.model.half()
# with torch.no_grad():
#     with torch.cuda.amp.autocast():
#         for (id,(lectureid,lecture)) in enumerate(lectures_dict.items()):
#             lecture_trimmed = lecture[:400]
#             generator_prompt_string = "Here is a lecture. Write a few questions for self study. \n\n###Lecture:\n\n" + lecture_trimmed + "\n\n###Questions:\n\n" + ""
#             generator_prompt = tokenizer(generator_prompt_string, return_tensors='pt')
#             generator_prompt = {k: v.to(device) for k, v in generator_prompt.items()}
#             #generate questions
#             output_tokens = model.generate(**generator_prompt, max_new_tokens=30, temperature=0.1, do_sample=True, top_k=50, top_p=0.95, return_dict_in_generate=False)
#             #decode the questions
#             output_string = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
#             output_string = output_string.replace(generator_prompt_string, "")
#             print(output_string)

#             evaluator_prompt = '''
# Below is an instruction that describes a task, paired with an input that provides further context. 
# Write a response that appropriately completes the request. 
# ### Instruction: Evaluate if the following question is a good question to ask after reading the following lecture.
# ### Input:

# ### Lecture: ''' + lecture_trimmed + '''
# ### Question: ''' + output_string + '''


# ### Response: This question is '''
#             # print(evaluator_prompt)

#             output = evaluator(evaluator_prompt, do_sample=True, min_length=2, max_new_tokens=10, temperature=0.01)
#             output_text = output[0]['generated_text']
#             output_text = output_text.replace(evaluator_prompt, "")
#             print(output_text)
#             # break

